In [2]:
# Imports
library(Seurat)
library(SingleCellExperiment)
library(SingleR)
library(celldex)    
library(ggplot2)
library(clustree)
library(gridExtra)
library(stringr) 
library(scDblFinder)
library(dplyr)
library(SeuratDisk)

# Change setting for visualizing plots within VSCODE
options(repr.plot.width=20, repr.plot.height=12)

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



In [4]:
# Function to use ScDblFinder to find doublets
remove_seurat_doublets <- function(seurat_object) {
    # To be able to use ScDBIFinder, we need to convert it to SCE
    sce_object <- as.SingleCellExperiment(seurat_object)
    
    # Run scDblFinder function
    sce_object <- scDblFinder(sce_object)

    # View table
    table(sce_object$scDblFinder.class)

    # as.Seurat needs logcounts but we dont want to calculate log counts, to trick we assume sce logcounts = counts
    logcounts(sce_object) <- assay(sce_object, "counts")

    # Convert SCE object back to Seurat
    seurat_object <- as.Seurat(sce_object)
    seurat_object <- subset(seurat_object, subset = scDblFinder.class == "singlet")
    seurat_object[["RNA"]] <- as(seurat_object[["RNA"]], "Assay5")
    return(seurat_object)
}

In [6]:
# Get mice data
# Set wd
setwd("../../../Data/R Objects/Raw Seurat R Objects")
list.files()
getwd()

# Read in data
Cochaine <- readRDS("Cochaine_raw_modified_orig.rds")
Vafadarnejad <- readRDS("Vafadarnejad_raw_modified_orig.rds")
Winkels <- readRDS("Winkels_raw_modified_orig.rds")

# For the following layers to work, we first need to JoinLayers of each individual object
# Merge layers
Cochaine[["RNA"]] <- JoinLayers(Cochaine[["RNA"]])
Vafadarnejad[["RNA"]] <- JoinLayers(Vafadarnejad[["RNA"]])
Winkels[["RNA"]] <- JoinLayers(Winkels[["RNA"]])

# Remove doublets from individual samples
# To add reproducability in the future, consider adding BPARAM = bp, where bp = Multicore(n, seed)
Cochaine <- remove_seurat_doublets(Cochaine)
Vafadarnejad <- remove_seurat_doublets(Vafadarnejad)
Winkels <- remove_seurat_doublets(Winkels)

# Winkels has gene names in caps, so we need to fix that 
rownames(Winkels) <- tolower(rownames(Winkels))
rownames(Winkels) <- str_to_title(rownames(Winkels))

# Change this later
rownames(Winkels) <- ifelse(grepl("^Mt-", rownames(Winkels)), tolower(rownames(Winkels)), rownames(Winkels))


# Merge the Seurat objects
mouse_samples <- merge(Cochaine, y=list(Vafadarnejad, Winkels))

# Merge layers
mouse_samples[["RNA"]] <- JoinLayers(mouse_samples[["RNA"]])
mouse_samples$species <- "Mouse"

[1] "all_mice_macrophages_annotated.rds"         
 [2] "Bashore_raw.rds"                            
 [3] "Cochaine_raw.rds"                           
 [4] "Cochaine_raw_modified_orig.rds"             
 [5] "Dib_and_Horstmann_macrophages_annotated.rds"
 [6] "Dib_macrophages_annotated.rds"              
 [7] "Dib_raw.rds"                                
 [8] "Fernandez_raw.rds"                          
 [9] "Horstmann_macrophages_annotated.rds"        
[10] "Horstmann_raw.rds"                          
[11] "Menno_raw.rds"                              
[12] "menno_raw_postdoublet.rds"                  
[13] "Vafadarnejad_raw.rds"                       
[14] "Vafadarnejad_raw_modified_orig.rds"         
[15] "Winkels_raw.rds"                            
[16] "Winkels_raw_modified_orig.rds"

[1] "c:/Users/phili/Desktop/BMW Jaar 3/Afstudeerscriptie/project_scriptie/Data/R Objects/Raw Seurat R Objects"

Warning message:
"Layer 'data' is empty"
Warning message:
"Layer 'scale.data' is empty"
Creating ~2061 artificial doublets...

Dimensional reduction

Evaluating kNN...

Training model...

iter=0, 374 cells excluded from training.

iter=1, 369 cells excluded from training.

iter=2, 370 cells excluded from training.

Threshold found:0.679

140 (5.4%) doublets called

Warning message:
"Assay RNA changing from Assay to Assay5"
Warning message:
"Layer 'data' is empty"
Warning message:
"Layer 'scale.data' is empty"
Creating ~1500 artificial doublets...

Dimensional reduction

Evaluating kNN...

Training model...

iter=0, 92 cells excluded from training.

iter=1, 86 cells excluded from training.

iter=2, 82 cells excluded from training.

Threshold found:0.655

36 (3.9%) doublets called

Warning message:
"Assay RNA changing from Assay to Assay5"
Warning message:
"Layer 'data' is empty"
Warning message:
"Layer 'scale.data' is empty"
Creating ~2833 artificial doublets...

Dimensional reduction



In [8]:
# Get human data
# Read datasets
horstmann <- readRDS("Horstmann_raw.rds")
fernandez <- readRDS("Fernandez_raw.rds")
menno <- readRDS("menno_raw_postdoublet.rds")
bashore <- readRDS("Bashore_raw.rds")
Dib <- readRDS("Dib_raw.rds")

# Dib RDS object was converted to v3 instead of v5 by schard
Dib <- UpdateSeuratObject(Dib)

# Split by patient
splitted_dib <- SplitObject(Dib, split.by = "patient")

# Assign patient objects 
splitted_dib$P1$orig.ident <- "Dib et al. (2022), P1"
splitted_dib$P2$orig.ident <- "Dib et al. (2022), P2"
splitted_dib$P3$orig.ident <- "Dib et al. (2022), P3"
splitted_dib$P5$orig.ident <- "Dib et al. (2022), P5"
splitted_dib$P6$orig.ident <- "Dib et al. (2022), P6"
splitted_dib$P7$orig.ident <- "Dib et al. (2022), P7"

# Merge the Seurat objects
dib <- merge(splitted_dib$P1, y=list(splitted_dib$P2, splitted_dib$P3, splitted_dib$P5, splitted_dib$P6, splitted_dib$P7))

Validating object structure

Updating object slots

Ensuring keys are in the proper structure

Ensuring keys are in the proper structure

Ensuring feature names don't have underscores or pipes

Updating slots in RNA

Validating object structure for Assay 'RNA'

Object representation is consistent with the most current Seurat version



In [ ]:
# Get human data (This step has to be chunked in 2 cell blocks, else my VSCode will crash)
# Removing Doublets
dib <- remove_seurat_doublets(dib)
fernandez <- remove_seurat_doublets(fernandez)
bashore <- remove_seurat_doublets(bashore)

Warning message in .checkSCE(sce, coerce = is.null(samples)):
"Some cells in `sce` have an extremely low read counts; note that these could trigger errors and might best be filtered out"
Warning message in scDblFinder(sce_object):
"You are trying to run scDblFinder on a very large number of cells. If these are from different captures, please specify this using the `samples` argument.TRUE"
Creating ~25000 artificial doublets...

Dimensional reduction

Evaluating kNN...

Training model...

iter=0, 6765 cells excluded from training.

iter=1, 7367 cells excluded from training.

iter=2, 7466 cells excluded from training.

Threshold found:0.424

4680 (13.6%) doublets called

Warning message:
"Assay RNA changing from Assay to Assay5"
Warning message:
"Layer 'data' is empty"
Warning message:
"Layer 'scale.data' is empty"
Creating ~4544 artificial doublets...

Dimensional reduction

Evaluating kNN...

Training model...

iter=0, 573 cells excluded from training.

iter=1, 526 cells excluded from 

In [10]:
# Get human data (This step has to be chunked, else my VSCode will crash)
# Removing Doublets
horstmann <- remove_seurat_doublets(horstmann)

Warning message:
"Layer 'data' is empty"
Warning message:
"Layer 'scale.data' is empty"
Warning message in scDblFinder(sce_object):
"You are trying to run scDblFinder on a very large number of cells. If these are from different captures, please specify this using the `samples` argument.TRUE"
Creating ~25000 artificial doublets...

Dimensional reduction

Evaluating kNN...

Training model...

iter=0, 8261 cells excluded from training.

iter=1, 8806 cells excluded from training.

iter=2, 8609 cells excluded from training.

Threshold found:0.388

5841 (15.8%) doublets called

Warning message:
"Assay RNA changing from Assay to Assay5"


In [12]:
# Merge the Seurat objects
human_samples <- merge(fernandez, y=list(horstmann, menno, bashore, dib)) #, dib_p1, dib_p2, dib_p3, dib_p5, dib_p6, dib_p7))

# Merge layers
human_samples[["RNA"]] <- JoinLayers(human_samples[["RNA"]], f = human_samples$orig.ident)
human_samples$species <- "Human"

Warning message:
"Some cell names are duplicated across objects provided. Renaming to enforce unique cell names."


In [13]:
# QC to remove low-quality cells
qc <- function(data, nFeatures_RNA = 200, nCounts_RNA = 3, percents.mt = 15, pattern = "MT") {
    # Split
    data[["RNA"]] <- split(data[["RNA"]], f = data$orig.ident)

    # Get percentage mt                      
    data[["percent.mt"]] <- PercentageFeatureSet(data, pattern = sprintf("^%s-", pattern))

    # Subsets
    data <- subset(x = data, subset = nFeature_RNA >= nFeatures_RNA & nCount_RNA >= nCounts_RNA & percent.mt <= percents.mt)

    # Rejoin layers
    data[["RNA"]] <- JoinLayers(data[["RNA"]])

    return(data)
}

# Removing low-quality cells
mouse_samples <- qc(mouse_samples, pattern= "mt")
human_samples <- qc(human_samples, pattern= "MT", percents.mt = 15)

Splitting 'counts', 'data' layers. Not splitting . If you would like to split other layers, set in `layers` argument.

Splitting 'counts', 'data' layers. Not splitting . If you would like to split other layers, set in `layers` argument.



In [16]:
# Convert to V3
mouse_samples[["RNA3"]] <- as(object = mouse_samples[["RNA"]], Class = "Assay")
DefaultAssay(mouse_samples) <- "RNA3"
mouse_samples[["RNA"]] <- NULL
mouse_samples <- RenameAssays(object = mouse_samples, RNA3 = 'RNA')

Warning message:
"No layers found matching search pattern provided"
Warning message:
"No layers found matching search pattern provided"
Warning message:
"Layer 'scale.data' is empty"
Warning message:
"Key 'rna_' taken, using 'rna3_' instead"
Renaming default assay from RNA3 to RNA

Warning message:
"Key 'rna3_' taken, using 'rna_' instead"


In [18]:
# Convert mice to h5ad
SaveH5Seurat(mouse_samples, filename = "mouse_samples2.h5Seurat")
Convert("mouse_samples2.h5Seurat", dest = "h5ad")

Creating h5Seurat file for version 3.1.5.9900

Adding counts for RNA

Adding data for RNA

No variable features found for RNA

Adding feature-level metadata for RNA

Validating h5Seurat file

Adding data from RNA as X

Transfering meta.features to var

Adding counts from RNA as raw

Transfering meta.features to raw/var

Transfering meta.data to obs



In [19]:
# Convert to V3
human_samples[["RNA3"]] <- as(object = human_samples[["RNA"]], Class = "Assay")
DefaultAssay(human_samples) <- "RNA3"
human_samples[["RNA"]] <- NULL
human_samples <- RenameAssays(object = human_samples, RNA3 = 'RNA')

Warning message:
"No layers found matching search pattern provided"
Warning message:
"No layers found matching search pattern provided"
Warning message:
"Layer 'scale.data' is empty"
Warning message:
"Key 'rna_' taken, using 'rna3_' instead"
Renaming default assay from RNA3 to RNA

Warning message:
"Key 'rna3_' taken, using 'rna_' instead"


In [20]:
# Convert human to h5ad
SaveH5Seurat(human_samples, filename = "human_samples.h5Seurat")
Convert("human_samples.h5Seurat", dest = "h5ad")

Creating h5Seurat file for version 3.1.5.9900

Adding counts for RNA

Adding data for RNA

No variable features found for RNA

Adding feature-level metadata for RNA

Validating h5Seurat file

Adding data from RNA as X

Transfering meta.features to var

Adding counts from RNA as raw

Transfering meta.features to raw/var

Transfering meta.data to obs



In [ ]:
###############################################################
"""
The integration steps are being processed in a subsequent Python notebook. 

"""